In [1]:
import math
from tensorboardX import SummaryWriter
import env_configurations
sess = None


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



In [2]:
import tensorflow as tf
import collections
gpu_options = tf.GPUOptions(allow_growth=True,per_process_gpu_memory_fraction=0.8)

config=tf.ConfigProto(gpu_options=gpu_options)
sess = tf.InteractiveSession(config=config)

In [3]:
import gym
import wrappers
import games_configurations
env_name = 'RoboschoolHumanoid-v1'#'RoboschoolHalfCheetah-v1'#'BipedalWalker-v2'#'Pendulum-v0'#'QuadruppedWalk-v1' #'QuadruppedWalk-v1' #$'BipedalWalker-v2'#'RoboschoolAnt-v1'
#'BipedalWalker-v2'#'RoboschoolAnt-v1' #'LunarLanderContinuous-v2'#'BipedalWalker-v2'#'LunarLander-v2' 
#'MountainCarContinuous-v0'#'CarRacing-v0'#'CartPole-v1' #'RoboschoolAnt-v1' #'CarRacing-v0' #'LunarLander-v2' #'Acrobot-v1' #
#a2c_config = games_configurations.pendulum_lstm_config
#a2c_config = games_configurations.halfcheetah_lstm_config_v2
#a2c_config = games_configurations.roboschoolant_config
#a2c_config = a2c_games_configurations.pendulum_config
#a2c_config = games_configurations.bipedalwalker_config
a2c_config = games_configurations.roboschoolhumanoid_lstm_config
#a2c_config = games_configurations.bipedalwalker_lstm_config
obs_space, action_space = env_configurations.get_obs_and_action_spaces(env_name)
print(obs_space)
print(action_space)


Box(88,)
Box(17,)


In [4]:
import ray
ray.init(redis_max_memory=1024*1024*100, object_store_memory=1024*1024*100)

2019-07-21 21:57:19,664	INFO node.py:469 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-07-21_21-57-19_6638/logs.
2019-07-21 21:57:19,779	INFO services.py:407 -- Waiting for redis server at 127.0.0.1:28203 to respond...
2019-07-21 21:57:19,925	INFO services.py:407 -- Waiting for redis server at 127.0.0.1:22744 to respond...
2019-07-21 21:57:19,934	INFO services.py:804 -- Starting Redis shard with 0.1 GB max memory.
2019-07-21 21:57:19,983	INFO node.py:483 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-07-21_21-57-19_6638/logs.
2019-07-21 21:57:19,985	INFO services.py:1427 -- Starting the Plasma object store with 0.1 GB memory using /dev/shm.


{'node_ip_address': '192.168.3.106',
 'redis_address': '192.168.3.106:28203',
 'object_store_address': '/tmp/ray/session_2019-07-21_21-57-19_6638/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2019-07-21_21-57-19_6638/sockets/raylet',
 'webui_url': None}

In [5]:
from a2c_continuous import A2CAgent
import tr_helpers
import networks



agent = A2CAgent(sess,'run', obs_space, False, action_space, a2c_config)
agent.restore('nn/latest_run')
agent.train()
#agent.save('nn/latest')

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Use tf.cast instead.
0
1


2019-07-21 21:57:21,579	ERROR worker.py:1672 -- WARNING: 24 workers have been started. This could be a result of using a large number of actors, or it could be a consequence of using nested tasks (see https://github.com/ray-project/ray/issues/3644) for some a discussion of workarounds.


2
3
4
5
6
7
8
9
10
11
12
13
14
15
0
Instructions for updating:
Use tf.cast instead.
(pid=6747) 
(pid=6747) WARNING: The TensorFlow contrib module will not be included in TensorFlow 2.0.
(pid=6747) For more information, please see:
(pid=6747)   * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
(pid=6747)   * https://github.com/tensorflow/addons
(pid=6747) If you depend on functionality not listed there, please file an issue.
(pid=6747) 
(pid=6748) 
(pid=6748) WARNING: The TensorFlow contrib module will not be included in TensorFlow 2.0.
(pid=6748) For more information, please see:
(pid=6748)   * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
(pid=6748)   * https://github.com/tensorflow/addons
(pid=6748) If you depend on functionality not listed there, please file an issue.
(pid=6748) 
(pid=6744) 
(pid=6744) WARNING: The TensorFlow contrib module will not be included in TensorFlow 2.0.
(pid=6744) For more informatio

INFO:tensorflow:Restoring parameters from nn/latest_run
Frames per seconds:  1379.198856299872
saving next best rewards:  63.542272705245445
Frames per seconds:  1837.0886845810198
saving next best rewards:  274.8275879496336
Frames per seconds:  1796.131198451008
saving next best rewards:  617.2239520555563
Frames per seconds:  1811.0233534188385
saving next best rewards:  917.9892458460032
Frames per seconds:  1787.1520250677547
Frames per seconds:  1799.6609304045496
saving next best rewards:  929.1711556078734
Frames per seconds:  1787.4195898517075
saving next best rewards:  993.4941314946486
Frames per seconds:  1735.2372590395642
Frames per seconds:  1785.671734077154
Frames per seconds:  1823.1430763279836
saving next best rewards:  1122.8155611600046
Frames per seconds:  1779.712878123236
saving next best rewards:  1186.090609842116
Frames per seconds:  1829.1467257279005
saving next best rewards:  1277.7817038310395
Frames per seconds:  1786.4016350170662
saving next best rew

Frames per seconds:  1798.1752865824435
Frames per seconds:  1731.5861030186675
Frames per seconds:  1796.069232200693
Frames per seconds:  1791.467355785519
Frames per seconds:  1779.495353234071
Frames per seconds:  1815.4376124552227
Frames per seconds:  1785.7027302022734
Frames per seconds:  1786.171701064943
Frames per seconds:  1792.3663583001019
Frames per seconds:  1773.0592010191763
Frames per seconds:  1769.7534455085365
Frames per seconds:  1805.0142357660725
Frames per seconds:  1706.5603025820017
Frames per seconds:  1811.9780236400595
Frames per seconds:  1811.641730462144
Frames per seconds:  1739.32530915781
Frames per seconds:  1802.583420114361
Frames per seconds:  1749.1576066128841
Frames per seconds:  1811.9361712674515
Frames per seconds:  1770.109747021778
Frames per seconds:  1758.6830193357307
Frames per seconds:  1779.4568310636282
Frames per seconds:  1811.8037470453821
Frames per seconds:  1773.8343180701197
Frames per seconds:  1785.3527406880103
Frames pe

...

Frames per seconds:  1761.9986492608318
Frames per seconds:  1823.2951592418474
Frames per seconds:  1763.8581666536277
Frames per seconds:  1833.8291737757443
Frames per seconds:  1821.0304740852564
Frames per seconds:  1733.4471871729581
Frames per seconds:  1811.3164474057387
Frames per seconds:  1783.4187696392962
Frames per seconds:  1775.6594494897008
Frames per seconds:  1784.9487346975584
Frames per seconds:  1790.1768749901007
Frames per seconds:  1712.324774069584
Frames per seconds:  1834.1968627094261
Frames per seconds:  1792.770362003581
Frames per seconds:  1794.4828511729406
Frames per seconds:  1845.1280498544936
Frames per seconds:  1766.4127712410182
Frames per seconds:  1814.9516159583848
Frames per seconds:  1760.8154076416076
Frames per seconds:  1824.2405382463755
Frames per seconds:  1789.2931117731384
Frames per seconds:  1776.6152206825232
Frames per seconds:  1812.5207506720515
Frames per seconds:  1798.690566748889
Frames per seconds:  1766.9681592587317
Fra

Frames per seconds:  1795.9535733057799
Frames per seconds:  1817.1510307280676
Frames per seconds:  1849.003218667713
Frames per seconds:  1800.0440882141477
Frames per seconds:  1798.6086519178937
Frames per seconds:  1810.4345881208874
Frames per seconds:  1803.2039950404164
Frames per seconds:  1854.7514693997146
Frames per seconds:  1773.418848695562
Frames per seconds:  1838.8253246065174
Frames per seconds:  1810.0834197730255
Frames per seconds:  1778.7867385158406
Frames per seconds:  1839.7553582211935
Frames per seconds:  1762.1978183660774
Frames per seconds:  1826.175004172172
Frames per seconds:  1818.4616527126723
Frames per seconds:  1789.2882665330765
Frames per seconds:  1804.8337117769972
Frames per seconds:  1852.0333911873586
Frames per seconds:  1800.24591504535
Frames per seconds:  1812.0680414580697
Frames per seconds:  1798.8811652985432
Frames per seconds:  1809.4805902237802
Frames per seconds:  1849.5504345992156
Frames per seconds:  1687.6384536163087
Frame

Frames per seconds:  1823.150621828564
Frames per seconds:  1839.5034099599054
Frames per seconds:  1774.1970286620717
Frames per seconds:  1843.35160182358
Frames per seconds:  1806.4811932624677
Frames per seconds:  1794.9750121485629
Frames per seconds:  1818.7465696040035
Frames per seconds:  1806.552618559073
Frames per seconds:  1798.2358924546263
Frames per seconds:  1802.7402263033343
Frames per seconds:  1803.2367384468114
Frames per seconds:  1803.6238821357463
Frames per seconds:  1852.0817086888305
Frames per seconds:  1780.4989916474478
Frames per seconds:  1854.237194576045
Frames per seconds:  1776.7837119611033
Frames per seconds:  1843.6328970572154
Frames per seconds:  1850.9329542375428
Frames per seconds:  1650.0511763715483
Frames per seconds:  1644.8710828695807
Frames per seconds:  1604.9172857820809
Frames per seconds:  1613.3662480083979
Frames per seconds:  1564.6177653408843
Frames per seconds:  1801.6297410855877
Frames per seconds:  1817.606860491898
Frames

Exception in thread ray_print_logs:
Traceback (most recent call last):
  File "/home/trrrrr/anaconda3/envs/rl/lib/python3.6/site-packages/redis/connection.py", line 185, in _read_from_socket
    raise socket.error(SERVER_CLOSED_CONNECTION_ERROR)
OSError: Connection closed by server.

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/trrrrr/anaconda3/envs/rl/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/home/trrrrr/anaconda3/envs/rl/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "/home/trrrrr/anaconda3/envs/rl/lib/python3.6/site-packages/ray/worker.py", line 1547, in print_logs
    msg = pubsub_client.get_message()
  File "/home/trrrrr/anaconda3/envs/rl/lib/python3.6/site-packages/redis/client.py", line 3135, in get_message
    response = self.parse_response(block=False, timeout=timeout)
  File "/home/trrrrr/anaconda3/envs/rl/lib/py

KeyboardInterrupt: 

In [6]:
agent.save('nn/latest_run')
ray.shutdown()

In [ ]:
import gym
gym.envs.registry.all()